In [3]:
import requests
import json
import pandas as pd
import ast
from jsonschema import validate
import geojson


In [5]:
#Get data on community areas 
r_places = requests.get('https://chicagohealth.herokuapp.com/api/v1/places')
places_data = r_places.json()
for result in places_data["community_areas"]:
    for key, val in ast.literal_eval(result["geometry"]).items(): 
        if type(key) == str:
            key.replace('\\', '') 
        if type(val) == str:
            val.replace('\\', '') 
    
    
                

data = geojson.dumps(places_data)
print(type(data))
with open('CHA_places.json', 'w') as data_file:
    data_file.write(data)
    data_file.close
    



<class 'str'>


AttributeError: 'str' object has no attribute 'is_valid'

In [3]:
#Get data on all resources in chicago
r_resources = requests.get('https://chicagohealth.herokuapp.com/api/v1/resources_all')
resources_data = r_resources.json()

In [12]:
CHAplaces = {
  "type": "FeatureCollection",
  "features": []}

for result in places_data["community_areas"]:
    CHAplaces["features"].append({
    "type": "Feature",
    "properties": {"name": result["name"], "slug": result["slug"]},
    "geometry": {
      "type": "GeometryCollection", 
      "geometries": [ 
        { "type": "Point",
          "coordinates": result["centroid"]},
        {"type": "MultiPolygon",
          "coordinates":ast.literal_eval(result["geometry"])["coordinates"]}]}})



data = geojson.dumps(CHAplaces)
print(type(data))
with open('CHA_places_test.json', 'w') as data_file:
    data_file.write(data)
    data_file.close
    


    

#neighborhoods = pd.DataFrame({'name': geo_name, 'centroid': centroid, 'geom_type': geom_type, 'geom_coord': geom_coord})
#neighborhoods.to_csv("./data/neighborhoods.csv")ast.literal_eval(result["geometry"])["coordinates"]

<class 'str'>


In [4]:
#Get 1990, 2000, 2010 data on life expectancy
r_le_1990 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/1990/life-expectancy')
r_le_2000 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2000/life-expectancy')
r_le_2010 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2010/life-expectancy')

In [5]:
le_1990_data = r_le_1990.json()
le_2000_data = r_le_2000.json()
le_2010_data = r_le_2010.json() 

In [6]:
#Convert to Pandas DF and then to CSV

geo_name, geo_id, ind_id, le, year = [],[],[],[], []
for result in le_1990_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
    year.append(1990)

for result in le_2000_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
    year.append(2000)

for result in le_2010_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
    year.append(2010)
le_CHA = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'life_exp': le, 'year': year})
le_CHA.to_csv("./data/CHA_le.csv", index = False)

In [7]:
#Get 2016 overall health status data
r_ohs_2016 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2014-2016/overall-health-status')
ohs_2016_data = r_ohs_2016.json()

geo_name, geo_id, ind_id, ohs_low_ci, ohs_hi_ci, wt_num, ohs_weight = [],[],[],[],[],[], []
for result in ohs_2016_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    le.append(result["number"])
    ohs_low_ci.append(result["lower_95ci_weight_percent"])
    ohs_hi_ci.append(result["upper_95ci_weight_percent"])
    wt_num.append(result["weight_number"])
    ohs_weight.append(result["weight_percent"])
ohs_2016 = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'ohs_low_ci': ohs_low_ci, 'ohs_hi_ci': ohs_hi_ci, 
                       'wt_num': wt_num, 'ohs_weight': ohs_weight})
ohs_2016['year'] = 2016
ohs_2016.to_csv("./data/ohs_2016.csv", index = False)

In [8]:
#Get 2000, 2010 infant mortality data
r_im_2000 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2000-2004/infant-mortality')
r_im_2010 = requests.get('https://chicagohealth.herokuapp.com/api/v1/topic_area/2010-2014/infant-mortality')
r_im_2000_data = r_im_2000.json()
r_im_2010_data = r_im_2010.json()

geo_name, geo_id, ind_id, im_crude, im_avg, year = [],[],[],[], [], []
for result in r_im_2000_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    im_crude.append(result["crude_rate"])
    im_avg.append(result["ave_annual_number"])
    year.append(2000)

for result in r_im_2010_data:
    geo_name.append(result["geo_group_name"])
    geo_id.append(result["id"])
    ind_id.append(result["indicator"]["id"])
    im_crude.append(result["crude_rate"])
    im_avg.append(result["ave_annual_number"])
    year.append(2010)
CHA_im = pd.DataFrame({'geo_name': geo_name, 'geo_id': geo_id, 'ind_id': ind_id, 'inf_mort_crude': im_crude, 'inf_mort_avg': im_avg, 'year': year})
CHA_im.to_csv("./data/CHA_im.csv", index = False)